In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df_original = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv', index_col=0)
train_df_original.head()

In [ ]:
test_df_original = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv', index_col=0)
test_df_original.head()

In [ ]:
train_df = train_df_original.copy()

y = train_df.pop('target')
X = train_df

for column in X.columns:
    X[column] = (X[column] - X[column].mean()) / (X[column].max() - X[column].min())
    
X

In [ ]:
test_df = test_df_original.copy()

for column in test_df.columns:
    test_df[column] = (test_df[column] - test_df[column].mean()) / (test_df[column].max() - test_df[column].min())
    
test_df

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=(14,)),
    layers.Dense(256, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(1)
])

In [ ]:
model.compile(
    loss='mae',
    optimizer='adam'
)

In [ ]:
earlystoppin = EarlyStopping(
    min_delta=0.01,
    patience=5,
    restore_best_weights=True
)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=4000,
    callbacks=(earlystoppin),
    epochs=50,
)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()

In [ ]:
final_pred = model.predict(test_df)

In [ ]:
submission = pd.DataFrame(final_pred, index=test_df_original.index, columns=['target'])
submission.to_csv('random_forest.csv')